In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import pandas as pd
import numpy as np
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from keras.layers import Bidirectional
import codecs
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

TLDR: Inspired by the Jeremy's lstm, i tried using the convolutional layer on the top of that.
Collectivley these type of codes are Long-term Recurrent Convolutional Networks or CNN-LSTM networks
Here are few links explaining its effectiveness 
https://machinelearningmastery.com/cnn-long-short-term-memory-networks/

https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py

https://yerevann.github.io/2016/06/26/combining-cnn-and-rnn-for-spoken-language-identification/

Adding the CNN on top of LSTM helps in the sense that CNN combined with pooling layers can bring out important temporal features devoid of any noise which the LSTM can use more effectively.
In the end, bidirectional LSTM will help in classifying the data.

Let's go to the code directly.

In [ ]:
train = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
test = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv")

In [ ]:
#Some data cleaning
train.comment_text = train.comment_text.apply(lambda x : re.sub(' u ', 'you', x))
train.comment_text = train.comment_text.apply(lambda x : re.sub('\nu ', 'you', x))
train.comment_text = train.comment_text.apply(lambda x : re.sub(' u\n', 'you', x))
train.comment_text = train.comment_text.apply(lambda x : re.sub("i'm", 'i am', x))
train.comment_text = train.comment_text.apply(lambda x : re.sub("fucksex", 'fuck sex', x))

test.comment_text = test.comment_text.apply(lambda x : re.sub(' u ', 'you', x))
test.comment_text = test.comment_text.apply(lambda x : re.sub('\nu ', 'you', x))
test.comment_text = test.comment_text.apply(lambda x : re.sub(' u\n', 'you', x))
test.comment_text = test.comment_text.apply(lambda x : re.sub("i'm", 'i am', x))
test.comment_text = test.comment_text.apply(lambda x : re.sub("fucksex", 'fuck sex', x))

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#Let's see the words which constitutes the toxic comments
def getwordcountdf(data, key):
    filtered=data[data[key]==1]
    sequence=[]
    tr_words=set(stopwords.words('english'))
    for x in filtered.comment_text:
        sequence+=text_to_word_sequence(x, 
                                        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                        lower=True,
                                        split=" ")
    filtered_words = [word for word in sequence if word not in tr_words]
    df= pd.DataFrame({'words': filtered_words})
    z= df.groupby('words').size().reset_index(name='counts')
    return z.sort_values('counts',ascending=False)


In [ ]:
obscene=getwordcountdf(train, 'obscene')
toxic=getwordcountdf(train, 'toxic')
severe_toxic=getwordcountdf(train, 'severe_toxic')
threat=getwordcountdf(train, 'threat')
insult=getwordcountdf(train, 'insult')
identity_hate=getwordcountdf(train, 'identity_hate')

#### Lets look at the various keywords which are most frequent in each category of toxic comments

In [ ]:
obscene.head(50).plot.bar(x='words', y='counts', figsize=(20,4))

In [ ]:
severe_toxic.head(50).plot.bar(x='words', y='counts', figsize=(20,4))

In [ ]:
threat.head(50).plot.bar(x='words', y='counts', figsize=(20,4))

In [ ]:
insult.head(50).plot.bar(x='words', y='counts', figsize=(20,4))

In [ ]:
identity_hate.head(50).plot.bar(x='words', y='counts', figsize=(20,4))

We can see that most frequent words do corresponds to the class they have been assigned to. 
These are really toxic words and we should get rid of these comments anyhow.


### Let's now remove the stopwords from the data.
#### I know its not good idea to remove the stopwords from a LSTM but it seems to be working better this way.


In [ ]:
def cleanupDoc(s):
    stopset = set(stopwords.words('english'))
    stopset.add('wikipedia')
    tokens =sequence=text_to_word_sequence(s, 
                                        filters="\"!'#$%&()*+,-˚˙./:;‘“<=·>?@[]^_`{|}~\t\n",
                                        lower=True,
                                        split=" ")
    cleanup = " ".join(filter(lambda word: word not in stopset, tokens))
    return cleanup

test.comment_text=test.comment_text.apply(cleanupDoc)
train.comment_text=train.comment_text.apply(cleanupDoc)

##### Standard tokenization of data to get the indexes of embedding matrix

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train.comment_text)
sequences = tokenizer.texts_to_sequences(train.comment_text)
test_sequence=tokenizer.texts_to_sequences(test.comment_text)

###### Pad the sequence so that we have the same length input, since keras supports same length input only

In [ ]:
data = pad_sequences(sequences, maxlen=150)

In [ ]:
t_data = pad_sequences(test_sequence, maxlen=150)

In [ ]:
### Load the pretrained glove vectors
print('Indexing word vectors.')
embeddings_index = {}
f = codecs.open('../input/glove6b300dtxt/glove.6B.300d.txt', encoding='utf-8')
for line in f:
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
vector_length=300
length=150
num_classes=6

In [ ]:
word_index = tokenizer.word_index
print('Preparing embedding matrix.')
# prepare embedding matrix
nb_words = min(200000, len(word_index))
notfound=[]
embedding_matrix = np.zeros((nb_words, vector_length))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        notfound.append(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

In [ ]:
embedding_matrix.shape

### Here is the keras model with CNN + LSTM

In [ ]:
#keras stuff now
import os
import csv
import codecs
from keras.layers import LSTM, Convolution1D,Convolution2D, Flatten, Dropout, Dense, Input, Conv1D, GRU, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.layers import Merge, merge, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, Embedding
#Using Pretrained Embedding Matrix

#### Since it is a multi label classification, we need to have 6 sigmoid units in the last layer rather than having a softmax layer.

In [ ]:
def getjmodel():
    inp = Input(shape=(length,))
    x = Embedding(nb_words, vector_length, weights=[embedding_matrix])(inp)
    x = Conv1D(256, 3, activation='relu')(x)
    x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    x = GlobalMaxPooling1D()(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
labels=train[['toxic','severe_toxic', 'obscene','threat','insult', 'identity_hate' ]]
data_train, data_test, y_train, y_test, comm_train, comm_trst =train_test_split(data, np.array(labels),train.comment_text, test_size=0.20, random_state=22)

In [ ]:
galaxyModel=getjmodel()
galaxyModel.fit(data_train, y_train, 1024, epochs=3, validation_data=(data_test, y_test))

### submission script

In [ ]:
preds=galaxyModel.predict(t_data)
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission[list_classes] = preds
sample_submission.to_csv('mysubmission.csv', index=False)

# Future tasks:
1. Add data augmentation
2. Improve model hyperparameters
3. Correct misspelled words. 

Thanks for looking at the kernel, hit the like button if you found it useful.